In [10]:
import matplotlib.pyplot as plt
from io import BytesIO
import xmltodict
import requests
from astropy.table import Table
import astropy.io.fits as fits

from ligo.gracedb.rest import GraceDb
g = GraceDb()

# O4a events

options to get all significant O4a mergers, or the 5 mergers considered in O4a paper

In [3]:
# use gracedb api call to get O4a significant superevents, but some of these have been retracted.

event_iterator = g.superevents('runid: O4a SIGNIF_LOCKED')
graceids = [superevent['superevent_id'] for superevent in event_iterator]

print (len(graceids), 'significant superevents in O4a')

89 significant superevents in O4a


In [4]:
responses = [g.superevent(id) for id in graceids]
data = [r.json() for r in responses]

In [14]:
def get_gcn_urls (ids, files):
    """
    remove retracted events
    get the most recent file for each superevent
    """ 

    superevent_files = [i['links']['files'] for i in files]

    event_files = [g.files(graceid).json() for graceid in ids]

    file = ['none' if any('etraction' in s for s in list(files))
            else id+'-5-Update.xml,0' if id+'-5-Update.xml,0' in list(files)
            else id+'-5-Update.xml' if id+'-5-Update.xml' in list(files)
            else id+'-4-Update.xml,0' if id+'-4-Update.xml,0' in list(files)
            else id+'-4-Update.xml' if id+'-4-Update.xml' in list(files)
            else id+'-3-Update.xml,0' if id+'-3-Update.xml,0' in list(files)
            else id+'-2-Update.xml,0' if id+'-2-Update.xml,0' in list(files)
            else id+'-4-Initial.xml,0' if id+'-4-Initial.xml,0' in list(files) 
            else id+'-3-Initial.xml,0' if id+'-3-Initial.xml,0' in list(files)
            else id+'-2-Initial.xml,0' if id+'-2-Initial.xml,0' in list(files)
            else id+'-2-Preliminary.xml,0' if id+'-2-Preliminary.xml,0' in list(files)
            else 'none' for files, id in zip(event_files, graceids)]

    urls = [i+j for i,j in zip(superevent_files, file)]

    [print(x) for x in urls if "none" in x]
    urls_save = [x for x in urls if "none" not in x]
    
    return(urls_save)

def get_params(xml_urls):
    """
    get superevent_id and skymap from event files
    """

    try:
        response = requests.get(xml_urls)
        dict=xmltodict.parse(response.text)
        superevent_id  = [item['@value'] for item in dict['voe:VOEvent']['What']['Param'] if item.get('@name') == 'GraceID'][0]
        skymap_url = [item['Param']['@value'] for item in dict['voe:VOEvent']['What']['Group'] if item.get('@name') == 'GW_SKYMAP'][0]
        classification = [item for item in dict['voe:VOEvent']['What']['Group'] if item.get('@name') == 'Classification']
        prob_bns = float([item['@value'] for item in classification[0]['Param'] if item.get('@name') == 'BNS'][0])  
        prob_nsbh = float([item['@value'] for item in classification[0]['Param'] if item.get('@name') == 'NSBH'][0])  
        skymap_response = requests.get(skymap_url)
        skymap_bytes = BytesIO(skymap_response.content)
        return superevent_id, skymap_bytes, prob_bns, prob_nsbh
    
    except:
        print (xml_urls)

In [7]:
#the printed urls are retracted events or my function failed to get the correct file

gcn_urls = get_gcn_urls (graceids, data)

https://gracedb.ligo.org/api/superevents/S231112ag/files/none
https://gracedb.ligo.org/api/superevents/S230830b/files/none
https://gracedb.ligo.org/api/superevents/S230808i/files/none
https://gracedb.ligo.org/api/superevents/S230715bw/files/none
https://gracedb.ligo.org/api/superevents/S230712a/files/none
https://gracedb.ligo.org/api/superevents/S230708bi/files/none
https://gracedb.ligo.org/api/superevents/S230622ba/files/none


In [8]:
#returns event id, skymap bytes, skymap table, prob bns, prob nsbh    

params = [get_params(url) for url in gcn_urls]

In [9]:
# get subset of mergers that are potential BNS or NSBH

ns_events = [i for i in params if i[2] + i[3] > 0.1]
ids = [i[0] for i in ns_events] 
print(f'{len(ns_events)} events with prob > 0.1 for NSBH + BNS: {ids}') 

3 events with prob > 0.1 for NSBH + BNS: ['S230731an', 'S230627c', 'S230529ay']


In [13]:
# save fits files

for event in ns_events:
    with fits.open(event[1]) as hdul:
        hdul.writeto(f'{event[0]}.fits', overwrite=True)

In [15]:
# mergers triggered on in the O4a paper (https://arxiv.org/abs/2405.12403)

O4a_urls = ['https://gracedb.ligo.org/api/superevents/S230518h/files/S230518h-4-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S230529ay/files/S230529ay-5-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S230627c/files/S230627c-4-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S230731an/files/S230731an-4-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S231113bw/files/S231113bw-4-Update.xml,0',
            ]

In [16]:
O4a_events = [get_params(url) for url in O4a_urls]

In [17]:
for event in O4a_events:
    with fits.open(event[1]) as hdul:
        hdul.writeto(f'{event[0]}.fits', overwrite=True)